In [1]:
import tensorflow as tf
import pandas as pd
import numpy as numpy
from sklearn import datasets


In [2]:
# load the data set
iris = datasets.load_iris()
iris_data_w_target = [];

# prepare data

# the data comes originally with all the examples ordered, so to get a meaningful test sample, we need to shuffle it

# add the target to the data to be able to shuffle and partition it later
for i in range(len(iris.data)):
    value = numpy.append(iris.data[i], iris.target[i])
    iris_data_w_target.append(value)

#print(iris_data_w_target)
feature_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'label']

df = pd.DataFrame(data = iris_data_w_target, columns = feature_names )

# shuffle our data
df = df.sample(frac=1).reset_index(drop=True)


In [3]:
# partition of our data
# we reserve the 20% of the total records for testing
test_len = (len(df) * 20)//100;
training_df = df[test_len:]
test_df = df[:test_len]

#print(test_df)

In [8]:
# train the classifier

my_feature_columns = [
    tf.contrib.layers.real_valued_column('SepalLength', dimension=1, dtype=tf.float32),
    tf.contrib.layers.real_valued_column('SepalWidth', dimension=1, dtype=tf.float32),
    tf.contrib.layers.real_valued_column('PetalLength', dimension=1, dtype=tf.float32),
    tf.contrib.layers.real_valued_column('PetalWidth', dimension=1, dtype=tf.float32)
]


# format data as required by the tensorflow framework
x = {
    'SepalLength': numpy.array(training_df['SepalLength']),
    'SepalWidth' : numpy.array(training_df['SepalWidth']),
    'PetalLength': numpy.array(training_df['PetalLength']),
    'PetalWidth' : numpy.array(training_df['PetalWidth'])
}

#neural network
classifier = tf.estimator.DNNClassifier(
       feature_columns=my_feature_columns,
       hidden_units=[5, 5],
       n_classes=3)


# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    y=numpy.array(training_df['label']).astype(int),
    num_epochs=None,
    shuffle=True)

# Train model.
classifier.train(input_fn=train_input_fn, steps=4000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_model_dir': 'C:\\Users\\_domine3\\AppData\\Local\\Temp\\tmp5gjzllj9', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000000121D6FD0>, '_master': '', '_train_distribute': None, '_keep_checkpoint_max': 5, '_task_id': 0, '_evaluation_master': '', '_is_chief': True, '_num_worker_replicas': 1, '_task_type': 'worker', '_tf_random_seed': None, '_log_step_count_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\_domin

In [9]:

# test accuracy of the model
# format data as required by the tensorflow framework
x = {
    'SepalLength': numpy.array(test_df['SepalLength']),
    'SepalWidth' : numpy.array(test_df['SepalWidth']),
    'PetalLength': numpy.array(test_df['PetalLength']),
    'PetalWidth' : numpy.array(test_df['PetalWidth'])
}

# Define the training inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    y=numpy.array(test_df['label']).astype(int),
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("Test Accuracy: ", accuracy_score)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-18-15:45:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\_domine3\AppData\Local\Temp\tmp5gjzllj9\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-18-15:45:46
INFO:tensorflow:Saving dict for global step 4000: accuracy = 1.0, average_loss = 0.03394517, global_step = 4000, loss = 1.0183551
Test Accuracy:  1.0


In [10]:
# We do some more manual testing
# format data as required by the tensorflow framework
x = {
    'SepalLength': numpy.array([5.0, 6.7, 7.4]),
    'SepalWidth' : numpy.array([3.5, 3.1, 2.8]),
    'PetalLength': numpy.array([1.3, 4.4, 6.1]),
    'PetalWidth' : numpy.array([0.3, 1.4, 1.9])
}

expected = numpy.array([0, 1, 2])

# Define the training inputs
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    num_epochs=1,
    shuffle=False)

predictions = classifier.predict(input_fn=predict_input_fn)

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('\nPrediction is "{}" (certainity {:.1f}%), expected "{}"'.format(class_id, 100 * probability, expec))
    


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\_domine3\AppData\Local\Temp\tmp5gjzllj9\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "0" (certainity 100.0%), expected "0"

Prediction is "1" (certainity 98.0%), expected "1"

Prediction is "2" (certainity 99.8%), expected "2"


In [11]:
# export the model

def serving_input_receiver_fn():
    serialized_tf_example = tf.placeholder(dtype=tf.string, shape=[None], name='input_tensors')
    receiver_tensors      = {'predictor_inputs': serialized_tf_example}
    feature_spec          = {'SepalLength': tf.FixedLenFeature([25],tf.float32),
                            'SepalWidth': tf.FixedLenFeature([25],tf.float32),
                            'PetalLength': tf.FixedLenFeature([25],tf.float32),
                            'PetalWidth': tf.FixedLenFeature([25],tf.float32)}
    features              = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


classifier.export_savedmodel(export_dir_base="stored_model", serving_input_receiver_fn  = serving_input_receiver_fn,as_text=True)



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Restoring parameters from C:\Users\_domine3\AppData\Local\Temp\tmp5gjzllj9\model.ckpt-4000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"stored_model\\temp-b'1529337262'\\saved_model.pbtxt"


b'stored_model\\1529337262'